In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install --pre higlass-python

## Synced heatmaps

In [ ]:
import higlass as hg

# Configure remote data sources (tilesets)
tileset1 = hg.remote(
    uid="CQMd6V_cRw6iCI_-Unl3PQ",
    server="https://higlass.io/api/v1/",
    name="Rao et al. (2014) GM12878 MboI (allreps) 1kb",
)

tileset2 = hg.remote(
    uid="QvdMEvccQuOxKTEjrVL3wA",
    server="https://higlass.io/api/v1/",
    name="Rao et al. (2014) K562 MboI (allreps) 1kb",
)

# Create a HeatmapTrack for each tileset
track1 = tileset1.track("heatmap")
track2 = tileset2.track("heatmap")

# Create two independent Views, one for each heatmap
view1 = hg.view(track1, width=6)
view2 = hg.view(track2, width=6)

# Lock zoom & location for each View
view_lock = hg.lock(view1, view2)

# Concatenate Views side-by-side, and apply synchronization lock
(view1 | view2).locks(view_lock)

In [ ]:
# Lock zoom only for each view
(view1 | view2).locks(zoom=view_lock)

In [ ]:
# Lock location only for each view
(view1 | view2).locks(location=view_lock)

In [ ]:
# Create additional views and synchronize (apply black to white color range)

bw_color_range = ["rgba(255,255,255,1)", "rgba(0,0,0,1)"]
view3 = hg.view(track1.opts(colorRange=bw_color_range), width=6)
view4 = hg.view(track2.opts(colorRange=bw_color_range), width=6)

# Create stacked view configuration and lock views by column
((view1 / view2) | (view3 / view4)).locks(
    hg.lock(view1, view2),
    hg.lock(view3, view4),
)

## Value scale syncing

In [ ]:
# Create Tracks

# creates a hg.Track without a tileset
t1 = hg.track("top-axis")

# Creates a hg.RemoteTileset object
remote_tileset = hg.remote(
    uid="CQMd6V_cRw6iCI_-Unl3PQ",
    server="https://higlass.io/api/v1/",  # optional, "http://higlass.io/api/v1/" default
    name="Rao et al. (2014) GM12878 MboI (allreps) 1kb",
)

# Tileset.track() creates a hg.Track object binding the parent Tileset
t2 = remote_tileset.track("heatmap").opts(valueScaleMax=0.5)

# Create Views

# Positional arguments for `hg.view` are overloaded. Keyword arguments are
# forwarded to the creation of the `hg.View`. Layout fields (`x`, `y`,
# `width`, `height`) may also be assigned.

# (1) Track objects (positioning guessed based on track type)
view1 = hg.view(t1, t2, width=6)

# (2) (Track, position) tuple
view2 = hg.view((t1, "top"), t2, width=6)

# (3) hg.Tracks object
view3 = hg.view(hg.Tracks(top=[t1], center=[t2]), width=6)

# (View, Track) tuples -> ScaleValueLock
scale_value_lock = hg.lock((view1, t2), (view2, t2))

(view1 | view2).locks(scale_value_lock)

## Remote heatmaps

In [ ]:
# Initialize data sources
tset1 = hg.remote(
    uid="CQMd6V_cRw6iCI_-Unl3PQ",
    name="Rao et al. (2014) GM12878 MboI (allreps) 1kb",
)

tset2 = hg.remote(
    uid="QvdMEvccQuOxKTEjrVL3wA",
    name="Rao et al. (2014) K562 MboI (allreps) 1kb",
)

# Create a track for each data source
t1 = tset1.track("heatmap", height=300)
t2 = tset2.track("heatmap", height=300)

# Create a custom DividedTrack and modify color scale
t3 = hg.divide(t1, t2).opts(
    colorRange=["blue", "white"],
    valueScaleMin=0.1,
    valueScaleMax=10,
)

# Set initial view domains
domain = (7e7, 8e7)
v1 = hg.view(t1, width=4).domain(x=domain)
v2 = hg.view(t2, width=4).domain(x=domain)
v3 = hg.view(t3, width=4).domain(x=domain)

(v1 | v3 | v2).locks(hg.lock(v1, v2, v3))

## Extract track from another view config

In [ ]:
# Load view config from URL
url = "https://gist.githubusercontent.com/manzt/c2c498dac3ca9804a2b8e4bc1af3b55b/raw/ee8426c9728e875b6f4d65030c61181c6ba29b53/example.json"
config = hg.Viewconf.from_url(url)

# Display viewconfig
config

In [ ]:
# Inspect the Viewconf tracks
for position, track in config.views[0].tracks:
    print(position)  # position in view layout
    print(track)  # python object from `higlass-schema`

In [ ]:
# Extract a specific track from the above Viewconf and modify its properties
gene_annotation_track = config.views[0].tracks.top[0].properties(height=200)

# Display track in isolation
hg.view(gene_annotation_track)

## Remote bigWig tiles
The next section requires `fuse` and `simple-httpfs` to be installed

In [ ]:
%pip install fusepy simple-httpfs clodius

In [ ]:
# set up FUSE
import pathlib

mount_dir = pathlib.Path.cwd() / "mnt"

if not mount_dir.is_dir():
    mount_dir.mkdir()

# mount directory and start fuse
hg.fuse.start(mount_dir.absolute())

In [ ]:
import hg

# get mounted path
bwpath = hg.fuse.path(
    f"http://hgdownload.cse.ucsc.edu/goldenpath/hg19/encodeDCC/"
    f"wgEncodeSydhTfbs/wgEncodeSydhTfbsGm12878InputStdSig.bigWig"
)

track = hg.bigwig(bwpath).track("horizontal-bar")
hg.view(track)

## Local cooler files

This section describes how to load cooler files that are on the same filesystem as the Jupyter notebook.

In [ ]:
tileset = hg.cooler("../test/data/Dixon2012-J1-NcoI-R1-filtered.100kb.multires.cool")
hg.view(tileset.track("heatmap"))

## Local bigWig files (with chromsizes)

### TODO:  inline `chromsizes` tilesets are not yet implemented

In [ ]:
chromsizes = [
    ("chr1", 249250621),
    ("chr2", 243199373),
    ("chr3", 198022430),
    ("chr4", 191154276),
    ("chr5", 180915260),
    ("chr6", 171115067),
    ("chr7", 159138663),
    ("chr8", 146364022),
    ("chr9", 141213431),
    ("chr10", 135534747),
    ("chr11", 135006516),
    ("chr12", 133851895),
    ("chr13", 115169878),
    ("chr14", 107349540),
    ("chr15", 102531392),
    ("chr16", 90354753),
    ("chr17", 81195210),
    ("chr18", 78077248),
    ("chr20", 63025520),
    ("chr19", 59128983),
    ("chr21", 48129895),
    ("chr22", 51304566),
    ("chrX", 155270560),
    ("chrY", 59373566),
    ("chrM", 16571),
]

bigwig_fp = (
    "../test/data/wgEncodeCaltechRnaSeqHuvecR1x75dTh1014IlnaPlusSignalRep2.bigWig"
)
ts = hg.bigwig(bigwig_fp, chromsizes=chromsizes)
cs = hg.ChromSizes(chromsizes)

view1 = View(
    [
        Track("top-axis"),
        Track("horizontal-bar", tileset=ts),
        Track("horizontal-chromosome-labels", position="top", tileset=cs),
    ]
)
display, server, viewconf = higlass.display([view1])

display

## Local bedlike data

### TODO:  inline `bedlike` tilesets are not yet implemented

In [ ]:
import hg

bed = [
    ["chr1", 1000, 2000, "item #1", ".", "+"],
    ["chr2", 3000, 3500, "item #1", ".", "-"],
]

chroms = [
    ["chr1", 2100],
    ["chr2", 4000],
]


data = bedtiles(bed, chroms)
track = Track(
    track_type="bedlike",
    position="top",
    height=50,
    data=data,
    options={"minusStrandColor": "red"},
)


d, s, v = higlass.display([[track]])
d

## Custom data

In [ ]:
import numpy as np

dim = 2000
I, J = np.indices((dim, dim))
data = -(J + 47) * np.sin(np.sqrt(np.abs(I / 2 + (J + 47)))) - I * np.sin(
    np.sqrt(np.abs(I - (J + 47)))
)

In [ ]:
import clodius.tiles.npmatrix
from hg.tilesets import LocalTileset

ts = hg.server.add(
    LocalTileset(
        uid="my-custom-tileset",
        info=lambda: clodius.tiles.npmatrix.tileset_info(data),
        tiles=lambda tids: clodius.tiles.npmatrix.tiles_wrapper(data, tids),
    )
)

hg.view(
    (hg.track("top-axis"), "top"),
    (hg.track("left-axis"), "left"),
    (ts.track("heatmap", height=250).opts(valueScaleMax=0.5), "center"),
)